In [1]:
import numpy as np
import pandas as pd

In [2]:
path = "Dataset"
pitstops = pd.read_csv(f"{path}/pit_stops.csv")
drivers = pd.read_csv(f"{path}/drivers.csv")
races = pd.read_csv(f"{path}/races.csv")
results = pd.read_csv(f"{path}/results.csv")
qualifying = pd.read_csv(f"{path}/qualifying.csv")
races = pd.read_csv(f"{path}/races.csv")
max(pitstops["raceId"])  - min(pitstops["raceId"])


269

In [3]:
results_s = results.query("raceId >= 841 and ( statusId == 1 or (statusId >= 11 and statusId <= 19))")

In [4]:
pitstops_count = pitstops.groupby(['raceId' , 'driverId']).count()['stop']

In [5]:
results_s = results_s.merge(pitstops_count,how="left" , on=['raceId' , 'driverId'])

results_s['pitstrategy'] = np.where(results_s['positionOrder'] <= results_s['grid'], 1 ,0 ) 

In [6]:
results_s.dropna(subset=['stop'], inplace=True)

In [7]:

first_10_in_grid = results_s.query('grid <= 10 and grid>0')

first_10_in_grid.dtypes

resultId             int64
raceId               int64
driverId             int64
constructorId        int64
number              object
grid                 int64
position            object
positionText        object
positionOrder        int64
points             float64
laps                 int64
time                object
milliseconds        object
fastestLap          object
rank                object
fastestLapTime      object
fastestLapSpeed     object
statusId             int64
stop               float64
pitstrategy          int32
dtype: object

In [8]:
a = first_10_in_grid.drop(['number','resultId','points','laps','time','milliseconds','rank','fastestLapTime','fastestLap','fastestLapSpeed','statusId','time','position','positionText','resultId'], axis=1)
a = a.merge(races[['raceId','circuitId','year']], how='left',on=['raceId'])
a = a.merge(qualifying[['raceId' , 'driverId','q1','q2','q3']],on=['raceId' , 'driverId'],how="left")

In [9]:

new_column_order = [col for col in a.columns if col != 'pitstrategy'] + ['pitstrategy'] 
a = a.reindex(columns=new_column_order)

In [10]:
count = [0 ,0 ,0 ]
for i in a.index.values:
    q3 = a.loc[i,'q3']
    # print(q3)
    if  ":" not in str(q3):
        a.loc[i,'q3'] = a.loc[i - 1,'q3']
        count[2] = count[2]+1
    q2 = a.loc[i,'q2']
    if ":" not in str(q2):
        a.loc[i,'q2'] = a.loc[i - 1,'q2']
        count[1] = count[1]+1
    q1 = a.loc[i,'q1']
    if ":" not in str(q1):
        a.loc[i,'q1'] = a.loc[i - 1,'q1']
        count[0] = count[0]+1
    


In [11]:
a

,raceId,driverId,constructorId,grid,positionOrder,stop,circuitId,year,q1,q2,q3,pitstrategy
0,841,20,9,1,1,2.0,1,2011,1:25.296,1:24.090,1:23.529,1
1,841,1,1,2,2,2.0,1,2011,1:25.384,1:24.595,1:24.307,1
2,841,808,4,6,3,2.0,1,2011,1:25.543,1:25.582,1:25.247,1
3,841,4,6,5,4,3.0,1,2011,1:25.707,1:25.242,1:24.974,1
4,841,17,9,3,5,3.0,1,2011,1:25.900,1:24.658,1:24.395,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2139,1110,1,131,3,4,3.0,13,2023,1:58.563,1:52.345,1:47.087,0
2140,1110,4,117,9,5,2.0,13,2023,1:58.834,1:52.751,1:47.843,1
2141,1110,847,131,8,6,1.0,13,2023,1:59.035,1:52.605,1:47.805,1
2142,1110,846,1,7,7,2.0,13,2023,1:59.981,1:52.252,1:47.669,1


In [12]:
def convert_to_time(value):
    value = str(value)
    print(value )
    parts = value.split(':')
    parts1 = parts[1].split(".")
    
    minutes = int(parts[0])
    seconds = int(parts1[0])
    milliseconds = int(parts1[1])
    return 60*1000*minutes + 1000*seconds + milliseconds



In [13]:
for column in ['q3','q2','q1']:
    a[column] = a[column].apply(convert_to_time)

1:23.529
1:24.307
1:25.247
1:24.974
1:24.395
1:24.779
1:25.599
1:27.066
1:34.870
1:35.200
1:36.124
1:35.179
1:36.251
1:35.802
1:36.820
1:34.974
1:36.809
1:34.463
1:33.706
1:34.421
1:34.670
1:35.145
1:35.119
1:35.119
1:36.190
1:36.203
1:25.049
1:25.454
1:25.851
1:25.595
1:25.574
1:25.982
1:26.659
1:26.296
1:26.296
1:26.646
1:21.181
1:21.961
1:21.996
1:20.981
1:21.964
1:21.964
1:22.599
1:22.471
1:22.952
1:13.556
1:14.483
1:13.997
1:14.019
1:15.280
1:15.766
1:13.838
1:13.014
1:13.429
1:13.864
1:14.085
1:13.217
1:13.814
1:36.975
1:37.454
1:37.163
1:37.380
1:37.535
1:37.645
1:38.231
1:38.231
1:38.231
1:38.240
1:30.516
1:30.431
1:30.399
1:32.376
1:31.124
1:32.209
1:31.933
1:31.929
1:30.134
1:30.442
1:30.079
1:30.216
1:30.910
1:32.010
1:31.263
1:32.482
1:32.187
1:20.024
1:19.815
1:20.365
1:19.978
1:20.474
1:20.350
1:21.098
1:21.445
1:21.445
1:48.298
1:49.376
1:51.251
1:50.552
1:50.256
1:52.303
1:51.121
1:22.275
1:22.777
1:22.841
1:22.725
1:23.777
1:23.188
1:23.188
1:44.381
1:44.804
1:44.732
1

In [14]:
a['stop'] = a['stop'].astype('int64')
a['pitstrategy']= a['pitstrategy'].astype('int64')

In [15]:
a.dtypes

raceId           int64
driverId         int64
constructorId    int64
grid             int64
positionOrder    int64
stop             int64
circuitId        int64
year             int64
q1               int64
q2               int64
q3               int64
pitstrategy      int64
dtype: object

In [16]:
a

,raceId,driverId,constructorId,grid,positionOrder,stop,circuitId,year,q1,q2,q3,pitstrategy
0,841,20,9,1,1,2,1,2011,85296,84090,83529,1
1,841,1,1,2,2,2,1,2011,85384,84595,84307,1
2,841,808,4,6,3,2,1,2011,85543,85582,85247,1
3,841,4,6,5,4,3,1,2011,85707,85242,84974,1
4,841,17,9,3,5,3,1,2011,85900,84658,84395,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2139,1110,1,131,3,4,3,13,2023,118563,112345,107087,0
2140,1110,4,117,9,5,2,13,2023,118834,112751,107843,1
2141,1110,847,131,8,6,1,13,2023,119035,112605,107805,1
2142,1110,846,1,7,7,2,13,2023,119981,112252,107669,1


In [17]:
import numpy as np
from sklearn.model_selection import KFold
import tensorflow as tf
from keras.callbacks import EarlyStopping
from keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [81]:
# for i in range(min(a['circuitId']) , max(a['circuitId'])):

b = a
X = b.drop(columns=['pitstrategy','stop','q1','q2','q3'])  # Assuming 'target' is the name of the target variable column
y = a['stop']
    # print( i , len(y))

In [82]:
y

0       2
1       2
2       2
3       3
4       3
       ..
2139    3
2140    2
2141    1
2142    2
2143    1
Name: stop, Length: 2144, dtype: int64

In [104]:
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
encoder = LabelEncoder()    
y_train_encoded = encoder.fit_transform(np.array(y).reshape(-1,1))

C:\Users\bpk_e\anaconda3\envs\eAssessmentBeerwulf\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [105]:
y = pd.DataFrame(y_train_encoded)
y[2]

KeyError: 2

In [98]:


k_folds = 2
kf = KFold(n_splits=k_folds, shuffle=False)

def create_model():
    model = tf.keras.Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(128, activation='sigmoid'),
        Dense(len(np.unique(y)), activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

all_loss = []
all_accuracy = []
training_histories = []  

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model = create_model()
    print(X_train)
    # early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    history = model.fit(X_train, y_train, epochs=50, batch_size=1,verbose=2,validation_split=0.33 )

    training_histories.append(history.history)
    predictions = model.predict(X_test)
    loss, accuracy = model.evaluate(X_test, y_test)
    all_loss.append(loss)
    all_accuracy.append(accuracy)
    print(predictions.reshape(1,-1))
    print(tf.math.confusion_matrix( y_test,  predictions))
    
avg_loss = np.mean(all_loss)
avg_accuracy = np.mean(all_accuracy)

print(f'Average Loss: {avg_loss}, Average Accuracy: {avg_accuracy}')

plt.figure(figsize=(18, 9))
for fold, history in enumerate(training_histories):
    plt.subplot(k_folds, k_folds, fold + 1)
    plt.plot(history['accuracy'], label='Training Acc')
    plt.plot(history['val_accuracy'], label='Validation Acc')
    plt.xlabel('Epoch')
    plt.ylabel('Acc')
    plt.title(f'Fold {fold+1} Loss')
    plt.legend()

plt.tight_layout()
plt.show()



      raceId  driverId  constructorId  grid  positionOrder  circuitId  year
1072     978       807              4     5              6          9  2017
1073     978        20              6     3              7          9  2017
1074     978       839             10     7              8          9  2017
1075     978       815             10     6              9          9  2017
1076     978       838              1     8             11          9  2017
...      ...       ...            ...   ...            ...        ...   ...
2139    1110         1            131     3              4         13  2023
2140    1110         4            117     9              5         13  2023
2141    1110       847            131     8              6         13  2023
2142    1110       846              1     7              7         13  2023
2143    1110       840            117    10              9         13  2023

[1072 rows x 7 columns]
Epoch 1/50


ValueError: in user code:

    File "C:\Users\bpk_e\anaconda3\envs\eAssessmentBeerwulf\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\bpk_e\anaconda3\envs\eAssessmentBeerwulf\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\bpk_e\anaconda3\envs\eAssessmentBeerwulf\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\bpk_e\anaconda3\envs\eAssessmentBeerwulf\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\bpk_e\anaconda3\envs\eAssessmentBeerwulf\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "C:\Users\bpk_e\anaconda3\envs\eAssessment

In [ ]:
training_histories[0].keys()

In [ ]:
k_folds = 3
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

def create_model():
    model = tf.keras.Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

all_loss = []
all_accuracy = []
training_histories = []  

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model = create_model()

    early_stopping = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)

    history = model.fit(X_train, y_train, epochs=250, batch_size=1,verbose=0,validation_split=0.33, callbacks = [early_stopping])

    training_histories.append(history.history)

    loss, accuracy = model.evaluate(X_test, y_test)
    all_loss.append(loss)
    all_accuracy.append(accuracy)
    print(model.predict(X_test).reshape(-1,1))
avg_loss = np.mean(all_loss)
avg_accuracy = np.mean(all_accuracy)

print(f'Average Loss: {avg_loss}, Average Accuracy: {avg_accuracy}')

plt.figure(figsize=(12, 6))

for fold, history in enumerate(training_histories):
    plt.subplot(k_folds, k_folds, fold + 1)
    plt.plot(history['accuracy'], label='Training Acc')
    plt.plot(history['val_accuracy'], label='Validation Acc')
    plt.xlabel('Epoch')
    plt.ylabel('Acc')
    plt.title(f'Fold {fold+1} Loss')
    plt.legend()

plt.tight_layout()
plt.show()